https://discuss.huggingface.co/t/gpt-2-trained-models-output-repeated/12962

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.train import main
from src.utils import prepare_data

data_filepath = "../../data/processed.csv"

In [3]:
trn_df, val_df = prepare_data(data_filepath, filter_args={"key": "character", "value": "bitjockey"})

In [4]:
trn_df.head(10)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
3837,[_puts head in hands_],[_nods_],Nothing specific. Just...pay attention to her ...,Do you have any advice?,Not likely to take it well.,Right.,Yeah...we don't want to give Nix the idea that...,Thank you
1626,[_narrows eyes_] I see.,[_shrugs_] It's mostly just s-supporting codeb...,_Interesting_...and what industry would that be?,Sango Dynamics,Okay...what company?,I'm...just a programmer. Not...not even for on...,[_to Thea_] That’s alright. It’s understandabl...,"[_small yelp_] Y-yes. Sorry. I-I get nervous, ..."
4094,[_texts_] What I meant was that I agree. We sh...,[*under her breath*] hopper what the fuck\n[*t...,[_texts_] ok,"[I defer to your opinion, Charlotte. This face...",[*texts*]...We should add that we want any inf...,[*texts*] And we need to figure out _exactly_ ...,[*texts*] OK but if we rush this he might turn...,[_texts_] we should do this asap. we don't hav...
287,Yeah,((maximum awkward)),_is still standing_,((is hopper still standing? bc charlotte is si...,...Are you doing okay?,(( 😂 )),_awkwardly pats Charlotte on the back_,(( this is the most awkward thing ever lol ))
1727,(( “I have found that the way of the samurai i...,[_A lot of the annotation are comments or ques...,[_The annotation is in kanji. The handwriting ...,[_Picks out the Japanese edition of Hagakure_],[_Thea is still in the library. She's engrosse...,Thanks. Uhm. Good talk. [_nods at both of them...,Library.,Hmm. Is there a copy around?
3841,[_fidgets_],[_sighs_] Alright. Fine.,...Alex.,[_shakes head_] It's nothing.,What has been?,[_softly_] It's just been so hard.,...what is it?,[_puts head in hands_]
4908,Well it's one hell of a coincidence his sprite...,So...what are we at? He's worse at social inte...,"No kidding? He messaged somebody, she showed u...",He knew her by *name*.\n*Kalahan growls that l...,"Yeah, which is _fucked_.","*Kalahan doesn't stop pacing, but he mutters j...",Like I said. Doesn't make _sense_,He seemed surprised that another technomancer ...
2452,[_texts_] well theyre not going after _me_...b...,the universe is insulting me,[_texts_] yeah..,r u serious? AFTER i wiped your data????,[_texts_] where 2 begin. Short version: we r d...,crazy with you is always entertaining for me,sure,[_texts_] do u want an update on my life bcuz ...
868,They take some time to work,....I thought the meds were helping?,[_softly again_] I was going to go somewhere a...,[*isn't sure what to say*],(( 😂😂😂😂)),((I think you accidentally a word)),[_sighs_] because you had your reasons. And be...,...How are you not? Medusa _broke into your ap...
2261,"[_goes back out, and knocks on Julia and Akari...","[_waves back, heads into her Not Sad Apartment_]","Later, Hopper. [_waves_]","Yes, we're going to go get you food now.",_bark?_,"[_smiles_] We could all use it, I think. I'll....",I should...let you get your stuff packed away....,[_nods_]


In [5]:
main(trn_df, val_df)

06/05/2022 01:19:30 - WARNING - src.train -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False


{'output_dir': 'output-medium', 'model_type': 'gpt2', 'model_name_or_path': 'microsoft/DialoGPT-medium', 'config_name': 'microsoft/DialoGPT-medium', 'tokenizer_name': 'microsoft/DialoGPT-medium', 'cache_dir': 'cached', 'block_size': 512, 'do_train': True, 'do_eval': True, 'evaluate_during_training': False, 'per_gpu_train_batch_size': 4, 'per_gpu_eval_batch_size': 4, 'gradient_accumulation_steps': 1, 'learning_rate': 5e-05, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_epochs': 3, 'max_steps': -1, 'warmup_steps': 0, 'logging_steps': 1000, 'save_steps': 3500, 'save_total_limit': None, 'eval_all_checkpoints': False, 'no_cuda': False, 'overwrite_output_dir': True, 'overwrite_cache': True, 'should_continue': False, 'seed': 42, 'local_rank': -1, 'fp16': False, 'fp16_opt_level': 'O1'}


Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/1241 [00:00<?, ?it/s]

06/05/2022 01:34:39 - INFO - src.train -   Saving model checkpoint to output-medium/checkpoint-3500
06/05/2022 01:34:42 - INFO - src.train -   Saving optimizer and scheduler states to output-medium/checkpoint-3500
06/05/2022 01:35:38 - INFO - src.train -    global_step = 3723, average loss = 1.7945089038264326
06/05/2022 01:35:38 - INFO - src.train -   Saving model checkpoint to output-medium
06/05/2022 01:35:42 - INFO - src.train -   Evaluate the following checkpoints: ['output-medium']
06/05/2022 01:35:45 - INFO - src.dataset -   Creating features from dataset file at cached
06/05/2022 01:35:45 - INFO - src.dataset -   Saving features into cached file cached/gpt2_cached_lm_512
06/05/2022 01:35:45 - INFO - src.train -   ***** Running evaluation  *****
06/05/2022 01:35:45 - INFO - src.train -     Num examples = 552
06/05/2022 01:35:45 - INFO - src.train -     Batch size = 4


Evaluating:   0%|          | 0/138 [00:00<?, ?it/s]

06/05/2022 01:35:53 - INFO - src.train -   ***** Eval results  *****
06/05/2022 01:35:53 - INFO - src.train -     perplexity = tensor(4.6761)


{'perplexity_': tensor(4.6761)}

In [7]:
from transformers import pipeline, Conversation, ConversationalPipeline

In [17]:
conversational_pipeline = pipeline("conversational", model="output-small", tokenizer="microsoft/DialoGPT-small")

In [18]:
conversation_1 = Conversation("Hello. How are you?")
conversation_2 = Conversation("How is Shiro doing?")

responses = conversational_pipeline([conversation_1, conversation_2])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [26]:
responses[0].generated_responses

["I'm doing alright."]

In [27]:
responses

[Conversation id: ccc61d84-e1da-468e-a33e-2e1d0effdf3f 
 user >> Hello. How are you? 
 bot >> I'm doing alright. ,
 Conversation id: c094e49f-553a-4799-9a86-7af3e2e4bd47 
 user >> How is Shiro doing? 
 bot >> [_shrugs_] I don't know. ]

In [28]:
conversation_1.add_user_input("That's good!")

In [29]:
conversation_1

Conversation id: ccc61d84-e1da-468e-a33e-2e1d0effdf3f 
user >> Hello. How are you? 
bot >> I'm doing alright. 
user >> That's good! 

In [32]:
conversational_pipeline(conversation_1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Conversation id: ccc61d84-e1da-468e-a33e-2e1d0effdf3f 
user >> Hello. How are you? 
bot >> I'm doing alright. 
user >> That's good! 
bot >> [_sighs_] 

In [34]:
conversation_1.generated_responses[-1]

'[_sighs_]'